<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/GAETry20210723.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!git clone https://github.com/YuehMintTai/GAETest20210722.git

fatal: destination path 'GAETest20210722' already exists and is not an empty directory.


In [51]:
import pandas as pd
import numpy as np
import flask 
import sklearn
import joblib
print('pandas==',pd.__version__)
print('numpy==',np.__version__)
print('flask==',flask.__version__)
print('sklearn==',sklearn.__version__)
print('joblib==',joblib.__version__)

pandas== 1.1.5
numpy== 1.19.5
flask== 1.1.4
sklearn== 0.22.2.post1
joblib== 1.0.1


In [ ]:
#!pip install flask_ngrok

In [28]:
df=pd.read_csv('samples.csv')
df['sex']='男'
df.loc[df['性別']==2,'sex']='女'
x=df[['sex','年齡','BSRS總分','網路成癮分數YDQ','家庭滿意度apgar']]
x=pd.get_dummies(data=x,drop_first=True)
y=df['自殺意念01'].astype(int).to_frame()
x_y=pd.concat([x,y],axis=1)

##區分training, validating, testing 組
x_y['remainder']=x_y.index%6
x_y['group']='train'
x_y.loc[x_y['remainder']==0,'group']='test'
x_y.loc[x_y['remainder']==1,'group']='valid'
x_y.loc[x_y['remainder']==2,'group']='valid'
x_test=x_y.loc[x_y['group']=='test'][['sex_男','年齡','BSRS總分','網路成癮分數YDQ','家庭滿意度apgar']]
x_valid=x_y.loc[x_y['group']=='valid'][['sex_男','年齡','BSRS總分','網路成癮分數YDQ','家庭滿意度apgar']]
x_train=x_y.loc[x_y['group']=='train'][['sex_男','年齡','BSRS總分','網路成癮分數YDQ','家庭滿意度apgar']]
y_test=x_y.loc[x_y['group']=='test'][['自殺意念01']]
y_valid=x_y.loc[x_y['group']=='valid'][['自殺意念01']]
y_train=x_y.loc[x_y['group']=='train'][['自殺意念01']]

,自殺意念01
178,0
179,1
183,0
184,1
185,0


In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import joblib
model_logit=LogisticRegression()
result_logit=model_logit.fit(x_train,y_train)
predict_logit=result_logit.predict_proba(x_test)
type(predict_logit[:,1])
AUC=roc_auc_score(y_test,pd.DataFrame(predict_logit[:,1]))
joblib.dump(model_logit,'model_logit.sav')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['model_logit.sav']

In [59]:
import joblib
load_model=joblib.load('model_logit.sav')
newCase=pd.DataFrame({'sex_男':[1],'年齡':[20],'BSRS總分':[20],'網路成癮分數YDQ':[20],'家庭滿意度apgar':[2]})
##predict_prob=result_logit.predict_proba(newCase)
predict_prob=load_model.predict_proba(newCase)
round(predict_prob[:,1].item(),4)

0.8397

In [71]:
###利用已建利好的model_logit.sav作prediction的flask網頁......
from flask_ngrok import run_with_ngrok
###
from flask import Flask, render_template, request
app=Flask(__name__,template_folder='templates')
####
run_with_ngrok(app)
####

import pandas as pd
import numpy as np
import joblib

def fun_predict(SEX,AGE,BSRS,YDQ,APGAR):
  newCase=pd.DataFrame({'sex_男':[SEX],'年齡':[AGE],'BSRS總分':[BSRS],'網路成癮分數YDQ':[YDQ],'家庭滿意度apgar':[APGAR]})
  load_model=joblib.load('model_logit.sav')
  predict_prob=load_model.predict_proba(newCase)
  return round(predict_prob[:,1].item(),4)

@app.route('/',methods=['POST','GET'])
def index():
  title='輸入觀察值:'
  SEX=1
  AGE=2
  BSRS=2
  YDQ=2
  APGAR=2
  PRED=fun_predict(SEX,AGE,BSRS,YDQ,APGAR)
  if request.method=='POST':
    SEX=int(request.form.get('SEX'))
    AGE=int(request.form.get('AGE'))
    BSRS=int(request.form.get('BSRS'))
    YDQ=int(request.form.get('YDQ'))
    APGAR=int(request.form.get('APGAR'))
    PRED=fun_predict(SEX,AGE,BSRS,YDQ,APGAR)
  return render_template('index.html',title=title,SEX=SEX,AGE=AGE,BSRS=BSRS,YDQ=YDQ,APGAR=APGAR,PRED=PRED)

##index()
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d5dcd8b7bb75.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Jul/2021 04:49:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2021 04:49:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Jul/2021 04:49:42] "POST / HTTP/1.1" 200 -


In [ ]:

load_model=joblib.load('model_logit.sav')
newCase=pd.DataFrame({'sex_男':[1],'年齡':[20],'BSRS總分':[20],'網路成癮分數YDQ':[20],'家庭滿意度apgar':[2]})
##predict_prob=result_logit.predict_proba(newCase)
predict_prob=load_model.predict_proba(newCase)
PREDICT=round(predict_prob[:,1].item(),4)

##以下空白##

In [ ]:
###單純只是show資料的flask網頁
from flask_ngrok import run_with_ngrok
###
from flask import Flask, render_template, request
app=Flask(__name__,template_folder='templates')
####
run_with_ngrok(app)
####

df=pd.read_csv('samples.csv')
i=15

@app.route('/',methods=['POST','GET'])
def index():
  title='指定筆數:'
  ID=i
  if request.method=='POST':
    ID=int(request.form.get('ID'))
  myDf=df.loc[ID][['年齡','性別','網路成癮分數YDQ','家庭滿意度apgar']]
  AGE=myDf['年齡']
  SEX=myDf['性別']
  YDQ=myDf['網路成癮分數YDQ']
  APGAR=myDf['家庭滿意度apgar']
  return render_template('index.html',title=title,ID=ID,AGE=AGE,SEX=SEX,YDQ=YDQ,APGAR=APGAR)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2cc20a614cce.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Jul/2021 13:10:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 13:10:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2021 13:10:38] "POST / HTTP/1.1" 200 -
[2021-07-23 13:10:46,342] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/range.py", line 355, in get_loc
    return self._range.index(new_key)
ValueError: 200 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/di

In [ ]:
df=pd.read_csv('samples.csv')
i=20
myDf=df.loc[i][['年齡','性別','網路成癮分數YDQ','家庭滿意度apgar']]
myDf['網路成癮分數YDQ']

1